# 🎨 Тестирование DreamBooth Моделей

**Простой способ протестировать обученную модель через AUTOMATIC1111 WebUI**

---

### 📋 Инструкция:
1. ▶️ Запустите все ячейки по порядку (Runtime → Run all)
2. 🔗 Откройте появившуюся ссылку Gradio
3. 🎨 Генерируйте изображения с вашим идентификатором

**Модель будет загружена из Google Drive автоматически!**

---

In [ ]:
#@title ⚙️ **НАСТРОЙКИ**
#@markdown Укажите ваш идентификатор (который использовали при обучении)

IDENTIFIER = "sksper"  #@param {type:"string"}
#@markdown Имя сессии (оставьте пустым для автоопределения: IDENTIFIER + "_model")
MODEL_NAME = ""  #@param {type:"string"}

# Если MODEL_NAME пустой - используем IDENTIFIER + "_model"
if MODEL_NAME == "":
    MODEL_NAME = IDENTIFIER + "_model"

# Настройка путей для сохранения изображений - в папку модели
OUTPUT_DIR = f"/content/gdrive/MyDrive/Fast-Dreambooth/Sessions/{MODEL_NAME}/generated-images"

print(f"✓ Идентификатор: {IDENTIFIER}")
print(f"✓ Имя модели: {MODEL_NAME}")
print(f"✓ Изображения будут сохраняться в: {OUTPUT_DIR}")
print("\n" + "="*60)
print("📝 ПРИМЕРЫ ПРОМПТОВ ДЛЯ ВАШЕГО ИДЕНТИФИКАТОРА:")
print("="*60)

print(f"\n🚺 Для женских фотографий:")
print(f"  • ({IDENTIFIER} person:1.2), (slim woman:1.2), professional portrait, studio lighting")
print(f"  • {IDENTIFIER} person, elegant dress, full body photo, natural pose, soft lighting")
print(f"  • ({IDENTIFIER} person:1.1), casual style, outdoor photography, golden hour")
print(f"  • {IDENTIFIER} person, business attire, confident pose, high quality portrait")

print(f"\n🚹 Для мужских фотографий:")
print(f"  • ({IDENTIFIER} person:1.2), (athletic man:1.2), professional headshot, sharp focus")
print(f"  • {IDENTIFIER} person, casual outfit, outdoor setting, natural lighting")
print(f"  • ({IDENTIFIER} person:1.1), formal suit, executive portrait, studio photography")
print(f"  • {IDENTIFIER} person, athletic pose, fitness photography, dynamic composition")

print(f"\n🎨 Универсальные:")
print(f"  • ({IDENTIFIER} person:1.2), professional portrait, detailed face, high quality")
print(f"  • {IDENTIFIER} person wearing sunglasses, street photography, urban background")
print(f"  • ({IDENTIFIER} person:1.1), artistic portrait, dramatic lighting, cinematic")

print(f"\n🚫 Рекомендуемый негативный промпт:")
print(f"  fat, overweight, ugly, deformed, bad anatomy, poorly drawn,")
print(f"  different skin tones, patchy skin, blurry, low quality")


In [ ]:
#@title 📁 **Подключение Google Drive**
from google.colab import drive
import os

print("\033[0;33mПодключение Google Drive...")
drive.mount('/content/gdrive')

# Проверка наличия модели
base_path = f"/content/gdrive/MyDrive/Fast-Dreambooth/Sessions/{MODEL_NAME}"

if os.path.exists(base_path):
    model_path = f"{base_path}/{MODEL_NAME}.ckpt"
    if not os.path.exists(model_path):
        model_path = f"{base_path}/{MODEL_NAME}.safetensors"
    
    if os.path.exists(model_path):
        print(f"\033[1;32m✓ Модель найдена: {model_path}")
    else:
        print(f"\033[1;31m✗ Файл модели не найден в {base_path}/")
        print("Доступные файлы:")
        !ls -lh "$base_path"
else:
    print(f"\033[1;31m✗ Сессия '{MODEL_NAME}' не найдена!")
    sessions_path = "/content/gdrive/MyDrive/Fast-Dreambooth/Sessions"
    if os.path.exists(sessions_path):
        print("\nДоступные сессии:")
        !ls -1 "$sessions_path"
    else:
        print("Папка Sessions не найдена. Убедитесь, что обучение завершено.")

In [ ]:
#@title 🔧 **Установка AUTOMATIC1111 WebUI**
import base64
import os

blsaphemy = base64.b64decode(("ZWJ1aQ==").encode('ascii')).decode('ascii')

print('\033[1;32mУстановка AUTOMATIC1111...')

# Устанавливаем в /content (локальное SSD, намного быстрее и надёжнее)
%cd /content

webui_path = f"stable-diffusion-w{blsaphemy}"
if not os.path.exists(webui_path):
    print("Клонирование репозитория...")
    !git clone -q --depth=1 --branch master https://github.com/AUTOMATIC1111/stable-diffusion-w$blsaphemy
    print("✓ Репозиторий клонирован")
else:
    print("✓ Репозиторий уже существует")

# Клонируем assets репозиторий
%cd /content/stable-diffusion-w$blsaphemy
!mkdir -p repositories
if not os.path.exists('repositories/stable-diffusion-w'+blsaphemy+'-assets'):
    !git clone -q https://github.com/AUTOMATIC1111/stable-diffusion-w$blsaphemy-assets repositories/stable-diffusion-w$blsaphemy-assets

# Скачиваем предсобранные репозитории (используем подход TheLastBen)
print("\n📦 Установка репозиториев Stable Diffusion...")
%cd /content

if not os.path.exists('/content/sd/stablediffusion'):
    print("Установка zstd...")
    !wget -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dependencies/zstd_1.4.4+dfsg-3ubuntu0.1_amd64.deb
    !dpkg -i zstd_1.4.4+dfsg-3ubuntu0.1_amd64.deb 2>/dev/null
    !rm zstd_1.4.4+dfsg-3ubuntu0.1_amd64.deb
    
    print("Скачивание архива репозиториев...")
    !wget -q https://huggingface.co/TheLastBen/dependencies/resolve/main/sd_mrep.tar.zst
    !tar -C /content --zstd -xf sd_mrep.tar.zst
    !rm sd_mrep.tar.zst
    print("✓ Репозитории распакованы")
else:
    print("✓ Репозитории уже существуют")

# Патчим paths.py чтобы он использовал наши репозитории
%cd /content/stable-diffusion-w$blsaphemy
!sed -i 's@possible_sd_paths =.*@possible_sd_paths = ["/content/sd/stablediffusion"]@' modules/paths.py
!sed -i 's@\.\.\/@src/@g' modules/paths.py
!sed -i 's@src/generative-models@generative-models@g' modules/paths.py

print('\n\033[1;32m✓ WebUI готов к запуску')


In [ ]:
#@title 📦 **Установка зависимостей**
print('\033[1;32mУстановка обязательных зависимостей...')

# Удаляем старый gradio (метод TheLastBen)
!rm -rf /usr/local/lib/python3.12/dist-packages/gradio* 2>/dev/null || true

# Скачиваем предсобранные зависимости от TheLastBen
%cd /content
if not os.path.exists('/usr/local/lib/python3.12/dist-packages/xformers'):
    print("Скачивание архива зависимостей...")
    !wget -q https://huggingface.co/TheLastBen/dependencies/resolve/main/gcolabdeps.tar.zst
    !tar -C / --zstd -xf gcolabdeps.tar.zst
    !rm gcolabdeps.tar.zst
    print("✓ Зависимости установлены")
else:
    print("✓ Зависимости уже установлены")

# Доустанавливаем недостающие пакеты
!pip install -q pydantic==1.10.2 2>/dev/null || true

# Применяем патчи для совместимости (метод TheLastBen)
print("Применение патчей совместимости...")
!sed -i 's@from pytorch_lightning.loggers.wandb import WandbLogger  # noqa: F401@@g' /usr/local/lib/python3.12/dist-packages/pytorch_lightning/loggers/__init__.py
!sed -i 's@globalns, localns, set()@globalns, localns, recursive_guard=set()@g' /usr/local/lib/python3.12/dist-packages/pydantic/typing.py

os.environ['PYTHONWARNINGS'] = 'ignore'
print('\033[1;32m✓ Все зависимости готовы')


In [ ]:
#@title 🔗 **Подготовка путей**
from google.colab import drive

# Проверка доступа к модели на Google Drive
if not os.path.exists(model_path):
    print("⚠️ Google Drive отключился, перемонтирую...")
    drive.flush_and_unmount()
    drive.mount('/content/gdrive', force_remount=True)
    print("✓ Google Drive перемонтирован")

# Проверяем что модель доступна
if os.path.exists(model_path):
    file_size = os.path.getsize(model_path) / (1024**3)
    print(f"\033[1;32m✓ Модель доступна: {os.path.basename(model_path)} ({file_size:.2f} GB)")
    print(f"  Путь: {model_path}")
else:
    print(f"\033[1;31m✗ ОШИБКА: Модель не найдена по пути {model_path}")
    raise FileNotFoundError("Модель не найдена!")

# Создаём директорию для сохранения изображений
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"\033[1;32m✓ Директория для изображений готова: {OUTPUT_DIR}")

print("\n💡 Оптимизация: Модель загружается напрямую с Google Drive (без копирования)")
print(f"   Экономия времени: ~30-60 секунд")
print(f"   Экономия места: ~{file_size:.1f} GB")


In [ ]:
#@title 🚀 **Запуск WebUI**
#@markdown После запуска откроется ссылка - нажмите на неё для доступа к интерфейсу
#@markdown ⏳ Первый запуск займёт 3-5 минут (установка зависимостей)

import json

print('\033[1;32mЗапуск AUTOMATIC1111 WebUI...')
print('⏳ Загрузка модели и запуск интерфейса...\n')

%cd /content/stable-diffusion-w$blsaphemy

# Создаём поддиректории для разных типов output
txt2img_dir = f"{OUTPUT_DIR}/txt2img"
img2img_dir = f"{OUTPUT_DIR}/img2img"
extras_dir = f"{OUTPUT_DIR}/extras"

os.makedirs(txt2img_dir, exist_ok=True)
os.makedirs(img2img_dir, exist_ok=True)
os.makedirs(extras_dir, exist_ok=True)

print(f"📁 Изображения будут сохраняться:")
print(f"  • txt2img: {txt2img_dir}")
print(f"  • img2img: {img2img_dir}")
print(f"  • extras:  {extras_dir}\n")

# Создаём конфигурационный файл для настройки output директорий
config_path = "config.json"
config = {}

# Проверяем существует ли config.json
if os.path.exists(config_path):
    try:
        with open(config_path, 'r') as f:
            config = json.load(f)
    except:
        config = {}

# Обновляем пути для сохранения изображений
config.update({
    "outdir_samples": "",
    "outdir_txt2img_samples": txt2img_dir,
    "outdir_img2img_samples": img2img_dir,
    "outdir_extras_samples": extras_dir,
    "outdir_grids": "",
    "outdir_txt2img_grids": f"{OUTPUT_DIR}/grids",
    "outdir_img2img_grids": f"{OUTPUT_DIR}/grids",
    "outdir_save": f"{OUTPUT_DIR}/saved"
})

# Сохраняем конфигурацию
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)

print(f"✓ Конфигурация сохранена в {config_path}\n")

# Прямой запуск с использованием модели с Google Drive
!python webui.py \
  --share \
  --api \
  --disable-safe-unpickle \
  --enable-insecure-extension-access \
  --no-download-sd-model \
  --no-half-vae \
  --skip-torch-cuda-test \
  --xformers \
  --ckpt "$model_path"


---

## 💡 Советы по использованию

### 📸 Примеры промптов:

**Позитивные промпты:**
```
{IDENTIFIER} person, professional portrait, studio lighting, high quality

({IDENTIFIER} person:1.2), elegant outfit, soft lighting, detailed face

{IDENTIFIER} person wearing sunglasses, street style, urban background

({IDENTIFIER} person:1.1), business attire, confident expression, sharp focus

{IDENTIFIER} person, casual clothing, outdoor setting, natural lighting, golden hour

({IDENTIFIER} person:1.2), artistic portrait, dramatic lighting, cinematic style
```

**Для женских фотографий:**
```
({IDENTIFIER} person:1.2), (slim woman:1.2), professional portrait, studio lighting

{IDENTIFIER} person, elegant dress, full body photo, natural pose, soft lighting

({IDENTIFIER} person:1.1), casual style, outdoor photography, golden hour
```

**Для мужских фотографий:**
```
({IDENTIFIER} person:1.2), (athletic man:1.2), professional headshot, sharp focus

{IDENTIFIER} person, casual outfit, outdoor setting, natural lighting

({IDENTIFIER} person:1.1), formal suit, executive portrait, studio photography
```

> Замените `{IDENTIFIER}` на ваш реальный идентификатор!

### 🚫 Негативные промпты (важно!):
```
fat, overweight, chubby, obese, ugly, deformed, bad anatomy, 
poorly drawn, different skin tones, patchy skin, uneven lighting,
blurry, low quality, worst quality, lowres, bad hands, mutation,
face swap, color mismatch
```

### ⚙️ Оптимальные параметры:
- **Sampling Steps**: 30-50 (больше = лучше качество)
- **CFG Scale**: 5-7 (ниже = естественнее, выше = точнее промпту)
- **Sampler**: DPM++ 2M Karras, Euler a, или DPM++ SDE Karras
- **Width/Height**: 512x512 (для SD 1.5)

### 🎨 Техники улучшения качества:

**Для лучшей согласованности кожи:**
- Используйте: `uniform skin tone, consistent lighting, natural colors`
- Избегайте высокого CFG (>7) - это создаёт артефакты

**Веса токенов:**
- `(slim:1.3)` - усиление признака
- `(fat:0.7)` - ослабление признака
- `(IDENTIFIER person:1.2)` - усиление идентификатора

**Для разных типов фото:**
- **Портрет**: `professional portrait, studio lighting, sharp focus`
- **Full body**: `full body photo, natural pose, uniform lighting`
- **Casual**: `casual photography, natural setting, soft lighting`
- **Artistic**: `artistic portrait, dramatic lighting, cinematic`

### 📌 Функции WebUI:
- 🎨 **txt2img** - генерация по описанию
- 🖼️ **img2img** - изменение существующих изображений  
- 🔧 **Extras** - улучшение качества (upscaling)
- ✏️ **Inpaint** - редактирование частей изображения

### 📁 Где найти изображения:
Все сгенерированные изображения сохраняются в папке вашей модели на Google Drive:
```
Google Drive → Fast-Dreambooth → Sessions → {MODEL_NAME} → generated-images/
```

### 🔄 Если результаты плохие:
1. **Снизьте CFG Scale** до 5-6
2. **Увеличьте Steps** до 40-50
3. **Попробуйте другой sampler** (Euler a обычно мягче)
4. **Улучшите промпт**: добавьте `uniform skin tone, consistent lighting`
5. **Рассмотрите переобучение** модели на более качественных данных

---
